## Подготовка данных

In [ ]:
# импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import statsmodels.api as sm # для построения линейной регрессии
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler # нормализация значений
from sklearn.model_selection import train_test_split # сплит на тестовую и тренировочную выборки
from sklearn.linear_model import LinearRegression # также для построения линейной регрессии
sns.set() # используем стили seaborn по умолчанию

In [ ]:
# импортируем спарсенные с Авито данные
df = pd.read_csv('avito_cars.csv')
df

In [ ]:
# Всегда полезно открыть в том числе и сам файл и бегло просмотреть данные
# Я обнаружил, что некоторые строки были занесены не корректно
df['Состояние'].value_counts()

In [ ]:
# как видим у нас 11 битых машин и 28 с некорретными значениями, можем просто удалить их из дата-сета
df_no_crash = df[df['Состояние'] == 'не битый']
df_no_crash

In [ ]:
# также заменяем владельцев 4+ на просто 4
df_no_crash = df_no_crash.replace({'Владельцев по ПТС': '4+'}, '4')

In [ ]:
df_no_crash.columns

In [ ]:
# удаляем ненужные нам столбцы
del df_no_crash['Unnamed: 0']
del df_no_crash['Состояние']

In [ ]:
# сбрасываем индексы
df_no_crash = df_no_crash.reset_index(drop=True)

In [ ]:
# преобразуем в числовые типы данных некоторые столбцы
df_no_crash['Год выпуска'] = df_no_crash['Год выпуска'].astype('int')
df_no_crash['Пробег'] = df_no_crash['Пробег'].astype('int')
df_no_crash['Владельцев по ПТС'] = df_no_crash['Владельцев по ПТС'].astype('int')
df_no_crash

## Анализ переменных

In [ ]:
# год выпуска
sns.distplot(df_no_crash['Год выпуска'])

In [ ]:
q = df_no_crash['Год выпуска'].quantile(0.01)
df_clean1 = df_no_crash[df_no_crash['Год выпуска']>q]
sns.distplot(df_clean1['Год выпуска'])

In [ ]:
# пробег
sns.distplot(df_clean1['Пробег'])

In [ ]:
q = df_clean1['Пробег'].quantile(0.99)
df_clean2 = df_clean1[df_clean1['Пробег']<q]
sns.distplot(df_clean2['Пробег'])

In [ ]:
# владельцы по ПТС
sns.distplot(df_clean2['Владельцев по ПТС'])

In [ ]:
# цена автомобиля
sns.distplot(df_clean2['Цена'])

In [ ]:
q = df_clean2['Цена'].quantile(0.99)
df_cleaned = df_clean2[df_clean2['Цена']<q]
sns.distplot(df_cleaned['Цена'])

## Предварительная оценка регрессии (statsmodel)

In [ ]:
y = df_cleaned['Цена']
x1 = df_cleaned[['Год выпуска', 'Пробег', 'Владельцев по ПТС']]

In [ ]:
x = sm.add_constant(x1)
results = sm.OLS(y,x).fit()
results.summary()

In [ ]:
df_cleaned.describe(include='all')

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize =(15,3)) #sharey -> сделать общей ординату y=price
ax1.scatter(df_cleaned['Год выпуска'],df_cleaned['Цена'])
ax1.set_title('Цена и год выпуска')
ax2.scatter(df_cleaned['Владельцев по ПТС'],df_cleaned['Цена'])
ax2.set_title('Цена и кол-во владельцев')
ax3.scatter(df_cleaned['Пробег'],df_cleaned['Цена'])
ax3.set_title('Цена и пробег')
plt.show()

In [ ]:
df_cleaned['Цена_лог'] = np.log(df_cleaned['Цена'])

In [ ]:
df_cleaned

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize =(15,3)) #sharey -> сделать общей ординату y=price
ax1.scatter(df_cleaned['Год выпуска'],df_cleaned['Цена_лог'])
ax1.set_title('Цена и год выпуска')
ax2.scatter(df_cleaned['Владельцев по ПТС'],df_cleaned['Цена_лог'])
ax2.set_title('Цена и кол-во владельцев')
ax3.scatter(df_cleaned['Пробег'],df_cleaned['Цена_лог'])
ax3.set_title('Цена и пробег')
plt.show()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
variables = df_cleaned[['Год выпуска','Владельцев по ПТС','Пробег']]
vif = pd.DataFrame()

In [ ]:
vif["ФИД"] = [variance_inflation_factor(variables.values, i) for i in range(variables.shape[1])]
vif["Переменные"] = variables.columns

In [ ]:
print(vif)

## Делаем из марок автомобилей фиктивные (dummy) переменные

In [ ]:
data_with_dummies = pd.get_dummies(df_cleaned, drop_first=True)
print(data_with_dummies.head())

In [ ]:
data_with_dummies.columns.values

In [ ]:
# убираем старую цену, она нам больше не нужна
data_with_dummies_clean = data_with_dummies.drop(['Цена'],axis=1)

In [ ]:
targets = data_with_dummies_clean['Цена_лог']
inputs = data_with_dummies_clean.drop(['Цена_лог'],axis=1)

## Линейная регрессия (sklearn)

In [ ]:
scaler = StandardScaler()
scaler.fit(inputs)
inputs_scaled = scaler.transform(inputs)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(inputs_scaled, targets, test_size=0.2, random_state=365)

In [ ]:
reg = LinearRegression()
reg.fit(x_train,y_train)

In [ ]:
y_hat = reg.predict(x_train)
plt.scatter(y_train, y_hat)
plt.xlabel('Targets (y_train)',size=18)
plt.ylabel('Predictions (y_hat)',size=18)
plt.xlim(11,15)
plt.ylim(11,15)
plt.show()

In [ ]:
sns.distplot(y_train - y_hat)
plt.title("Функция плотности вероятности остатков", size=18)
plt.show()

In [ ]:
reg.score(x_train,y_train)

In [ ]:
reg.intercept_

In [ ]:
reg_summary = pd.DataFrame(inputs.columns.values, columns=['Переменные'])
reg_summary['Веса'] = reg.coef_
print(reg_summary)

In [ ]:
y_hat_test = reg.predict(x_test)

In [ ]:
x_test

In [ ]:
plt.scatter(y_test, y_hat_test, alpha=0.2)
plt.xlabel('Targets (y_test)',size=18)
plt.ylabel('Predictions (y_hat_test)',size=18)
plt.xlim(12,15)
plt.ylim(12,15)
plt.show()

In [ ]:
x

In [ ]:
x = sm.add_constant(inputs)
results = sm.OLS(targets,x).fit()
results.summary()

In [ ]:
y_test = y_test.reset_index(drop=True) # не забываем дропнуть индексы, потому что наш дата сет был "перемешан"
df_pf = pd.DataFrame(np.exp(y_hat_test), columns=['Prediction']) # np.exp отвечает за преобразование логарифма
df_pf['Целевое значение'] = np.exp(y_test)
df_pf['Разница'] = df_pf['Target'] - df_pf['Предсказанное значение']
df_pf['Разница в %'] = np.absolute(df_pf['Разница']/df_pf['Целевое значение']*100)
print(df_pf)

In [ ]:
df_pf.to_excel('prediction.xls')

In [ ]:
polo = -195.7369 + 0.1043*2015 + 62000*(-0.00000089) - 1.1371
np.exp(polo)